# IOTA 101: Transactions
A transaction is a smallest piece of data that can be broadcasted to the network. Each transaction consists of fixed 2673 trytes. Transactions (TXs) are basically of two types:
* **Value TXs:** those transactions change ownership of IOTA tokens. Two possible states: spending TXs or receiving TXs
* **Non-Value TXs:** those transactions include piece of data that can be broadcasted and read from the Tangle

Transactions are broadcasted in an envelope called *Bundle*. Bundle may consist of multiple TXs. Bundle is processed by the Tangle as a single entity; meaning only the whole bundle is confirmed by the network (and all TXs within) or nothing. You have to prepare a bundle even if you broadcast single transaction only. Bundle is also described as *atomic* since every bundle includes all information needed to process it antonomously.

Each transaction includes several fields, let's name the most important ones:
* **`Value`:** Value can be *positive* number = receiving IOTA tokens, *negative* number = spending IOTA tokens or *zero* = broadcasting only piece of data
* **`Address`:** It is IOTA address that is associated with the given transaction. *Value* indicates whether it is an address of a receiver or a sender
* **`Tag`:** It is an user-defined tag that can be used for searching, etc.
* **`SignatureMessageFragment`:** This is the most lenghty attribute (2187 trytes) and could include: *transaction signature* based on private key in case it is a spending TX or *piece of data (message)* in case it is non-value TX. Alternatively it can be also left blank (in case of non-value TXs)
* **`tTrunkTransaction` / `branchTransaction`:** Those two fields refer to some previous transactions in the Tangle (tips) that the given bundle is going to approve
* **`Timestamp`:** TX timestamp. It is actually UNIX timestamp (UTC)
![Creating TX in 5 steps](https://raw.githubusercontent.com/Hribek25/IOTA101/master/Graphics/Creating_TX_in_5_steps.png "By Petr Zizka")

A process of sending IOTA transaction can be summarized in 5 steps:
* To create transation(s) with the given attributes: *value*, *address* and potentially also *tag* or *message*
* To finalize a bundle that includes all transactions. Once the bundle is finalized no new transactions can be added. It basically means to generate a *bundle hash*
* To search for two tips in the Tangle that you are going to validate. There is not a strict rule how to search them but it is generally recommended to leave it on a *tip selection algorithm* implemented on node's side 
* To do a *Proof-of-Work* for each transaction in the bundle. Result of POW is *Nonce* and *Transaction hash* stored with each transaction
* Finally you need to *broadcast* the whole bundle to the network

### A secret of a bundle
There is a secret to be shared. You have to understand one important thing. Bundle is a top level construct that link all related transactions under one entity however the bundle itself is not broadcasted in fact. You still broadcast "only" collection of individual transactions instead. All transactions are recognized to be part of a bundle by the IOTA protocol however any data peering is based on individual transactions (in trytes).

In other words, a bundle can be reconstructed anytime from a collection of transactions via fields `bundle hash`, `current index` and `last index`.

## Non-value transactions
Let's start with non-value transactions (meta transactions) first since they are a bit easier for starters.

*Please note, the chapter is divided in two separate sections. In the first section, examples are really close to bare metal (still not too close:) to illustrate the whole process and describing implementation details wherever possible. It is also useful when you want to experiment with different settings at each step (hey "non-existing tips", looking at you).*

*In the second section, PyOTA library is fully levearged since it is capable to hide all implementation details behind a curtain. Needless to say, if you are not interested in such details then you probably would like to start with the second section.*

### Creating and broadcasting transaction in 5 steps (you want to understand the process)
#### Create a transaction
First of all let's create transactions (offline; in memory) and set main fields.

BTW: `Timestamp` field is defined automatically by the PyOTA library and so please make sure it is also defined while using other IOTA-facing libraries. It is an important field while creating `bundle hash`.

In the examples within this chapter we are going to broadcast meta transactions (data only) to two different IOTA addresses.

In [2]:
import iota
from datetime import datetime
from pprint import pprint

MySeed = b"HGW9HB9LJPYUGVHNGCPLFKKPNZAIIFHZBDHKSGMQKFMANUBASSMSV9TAJSSMPRZZU9SFZULXKJ9YLAIUA"
TargetAddress1 = b"CXDUYK9XGHC9DTSPDMKGGGXAIARSRVAFGHJOCDDHWADLVBBOEHLICHTMGKVDOGRU9TBESJNHAXYPVJ9R9"
TargetAddress2 = b"CYJV9DRIE9NCQJYLOYOJOGKQGOOELTWXVWUYGQSWCNODHJAHACADUAAHQ9ODUICCESOIVZABA9LTMM9RW"

NowIs = datetime.now() # get a actual date & time - just to have some meaningfull info

# preparing transactions
pt = iota.ProposedTransaction(address = iota.Address(TargetAddress1), # 81 trytes long address
                              message = iota.TryteString.from_unicode('Here comes a first message. Now is %s' % (NowIs)),
                              tag     = iota.Tag(b'HRIBEK999IOTA999TUTORIAL'), # Up to 27 trytes
                              value   = 0)

pt2 = iota.ProposedTransaction(address = iota.Address(TargetAddress2), # 81 trytes long address
                               message = iota.TryteString.from_unicode('Here comes a second message. Now is %s' % (NowIs)),
                               tag     = iota.Tag(b'HRIBEK999IOTA999TUTORIAL'), # Up to 27 trytes
                               value   = 0)
# besides the given attributes, library also adds a transaction timestamp

print("Created transaction objects:\n")
pprint(vars(pt))
print("\n")
pprint(vars(pt2))

Created transaction objects:

{'_legacy_tag': None,
 'address': Address(b'CXDUYK9XGHC9DTSPDMKGGGXAIARSRVAFGHJOCDDHWADLVBBOEHLICHTMGKVDOGRU9TBESJNHAXYPVJ9R9'),
 'attachment_timestamp': 0,
 'attachment_timestamp_lower_bound': 0,
 'attachment_timestamp_upper_bound': 0,
 'branch_transaction_hash': TransactionHash(b'999999999999999999999999999999999999999999999999999999999999999999999999999999999'),
 'bundle_hash': None,
 'current_index': None,
 'hash': None,
 'is_confirmed': None,
 'last_index': None,
 'message': TryteString(b'RBTCFDTCEARCCDADTCGDEAPCEAUCXCFDGDHDEAADTCGDGDPCVCTCSAEAXBCDKDEAXCGDEAWAUAVABBRAUA9BRAWAABEAUA9BDBWAZADBZACBSAXAVAABXAABWA'),
 'nonce': Nonce(b'999999999999999999999999999'),
 'signature_message_fragment': None,
 'tag': Tag(b'HRIBEK999IOTA999TUTORIAL999'),
 'timestamp': 1530080759,
 'trunk_transaction_hash': TransactionHash(b'999999999999999999999999999999999999999999999999999999999999999999999999999999999'),
 'value': 0}


{'_legacy_tag': None,
 'address': Address(b

#### Finalizing bundle
Once all individual transactions are created it is time to prepare and finalize the bundle. While preparing the bundle you need at least to specify list of prepared transaction(s). In case of non-value transactions it is quite straightforward process. However, it is a bit more complicated in case of value transactions - please see later.

Finalizing the bundle consists of several tasks under the hood:
* Each transaction is indexed. Attributes `current_index` / `last_index` are set
* `Bundle hash` is generated (*Sponge function* + normalization) and assigned to each transaction
* `signatureMessageFragment` is bascially copy of `message` field in case of non-value transactions
* PyOTA library also checks whether the data in `message` field is larger than transaction allows (2187 trytes). If this is the case then it takes care of it and split your data into several transactions

*Please note, finalizing the bundle also means you are no longer able to add new transactions to the bundle post the finalization process.*

Simply put, `bundle hash` is a cryptographic "fingerprint" of all transactions in the bundle. It uniquely represents them and so as long as transactions are the same (incl their particular order) the `bundle hash` is also the same.

$$BundleHash = fce(address, value, legacy tag, timestamp, current index, last index)$$

You may be wondering what is a difference between `tag` and `legacy_tag`. `Tag` includes actualy the tag that was defined during the transaction creation. `Legacy_tag` is also based on it however it is modified during the normalization process while bundle hashing to be sure that `bundle hash` is can be securely used while TX signing. That's why the `bundle hash` is sometimes refered as `normalized bundle hash`.

The bundle refers to the first transaction in collection as `tail_transaction`. You can perceive `tail transaction` as an official representative of the whole bundle while checking for confirmation status, etc. Remember, the bundle is an atomic entity and so whatever is done to bundle it applies to ALL transactions within.

In [5]:
# preparing bundle that consists of both transactions prepared in the previous example
pb = iota.ProposedBundle(transactions=[pt,pt2]) # list of prepared transactions is needed at least

# generate bundle hash using sponge/absorb function + normalize bundle hash + copy bundle hash into each transaction / bundle is finalized
pb.finalize()

#bundle is finalized, let's print it
print("\nGenerated bundle hash: %s" % (pb.hash))
print("\nTail Transaction in the Bundle is a transaction #%s." % (pb.tail_transaction.current_index))
        
print("\nList of all transactions in the bundle:\n")
for txn in pb:
    pprint(vars(txn))
    print("")



Generated bundle hash: TL9EHBGBIDCTLCARLPKBSUQIG9CKQPDGTLOVNLG9TALAGEWQRBEK9DPLDDBGOAZIREKGYPYFCWJCTOADW

Tail Transaction in the Bundle is a transaction #0.

List of all transactions in the bundle:

{'_legacy_tag': Tag(b'PVIBEK999IOTA999TUTORIAL999'),
 'address': Address(b'CXDUYK9XGHC9DTSPDMKGGGXAIARSRVAFGHJOCDDHWADLVBBOEHLICHTMGKVDOGRU9TBESJNHAXYPVJ9R9'),
 'attachment_timestamp': 0,
 'attachment_timestamp_lower_bound': 0,
 'attachment_timestamp_upper_bound': 0,
 'branch_transaction_hash': TransactionHash(b'999999999999999999999999999999999999999999999999999999999999999999999999999999999'),
 'bundle_hash': BundleHash(b'TL9EHBGBIDCTLCARLPKBSUQIG9CKQPDGTLOVNLG9TALAGEWQRBEK9DPLDDBGOAZIREKGYPYFCWJCTOADW'),
 'current_index': 0,
 'hash': None,
 'is_confirmed': None,
 'last_index': 1,
 'message': TryteString(b'RBTCFDTCEARCCDADTCGDEAPCEAUCXCFDGDHDEAADTCGDGDPCVCTCSAEAXBCDKDEAXCGDEAWAUAVABBRAUA9BRAWAABEAUA9BDBWAZADBZACBSAXAVAABXAABWA'),
 'nonce': Nonce(b'999999999999999999999999999'),
 'signat

At this stage you can also see how does our finalized bundle look encoded in Trytes so far. It is also a proof that there is no bundle to be broadcasted itself, only list of transations matter.

In [6]:
Trytes = pb.as_tryte_strings() # bundle as trytes
pprint(Trytes)


[TransactionTrytes(b'RBTCFDTCEARCCDADTCGDEAPCEAGDTCRCCDBDSCEAADTCGDGDPCVCTCSAEAXBCDKDEAXCGDEAWAUAVABBRAUA9BRAWAABEAUA9BDBWAZADBZACBSAXAVAABXAABWA999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999

#### Selecting two tips
As mentioned earlier you need to find two tips that you are going to validate together with your bundle for a sake of network participation. This will be outsourced to an IOTA node and so it will be for the first time we interact with the network (via API call `get_transactions_to_approve`).

There is a `depth` parameter needed. It instructs a node how many milestones it should go in the past while confirming tips. The higher value the better to network however more resources needed. `Depth=3` is considered to be well-balanced compromise. Higher value than `depth=10` will probably throw an exception depending on node's configuration.

It should return two selected tips as `branch` and `trunk` transactions. Those transactions will be used while broadcasting the bundle.

In [7]:
api = iota.Iota("https://field.carriota.com:443") # selecting IOTA node via Field service

gta = api.get_transactions_to_approve(depth=3) # get tips to be approved by your bundle
pprint(gta)



{'branchTransaction': TransactionHash(b'KQBDDBHWHVLQSJFENGWRBHIOULOMC9GXDW99ECMVFHZXGFZQJOXATSMDLYQJFBZCXA9IJ9SQUXAFZ9999'),
 'duration': 1700,
 'fieldName': 'F-Node Montabaur',
 'fieldPublicId': '1bdeeb759f960554',
 'fieldVersion': '0.1.6',
 'trunkTransaction': TransactionHash(b'RRYDZLPS9IWCSWRROPQQJTKBRONOJNLUHVHFXFEOZAVGFEOUAHXWWHJSHAGWSSNSMWHN9SEYNBOEA9999')}


#### Performin' POW
*Proof of work* is a relatively simple cryptograhic puzzle to be solved. It represents energy-based costs of your transaction. It also helps to minimize risks of some attack vectors to the network.

This task can be also outsourced to IOTA nodes. In order to perform it you need *selected tips*, *finalized bundle* (in trytes) and also `Minimum Weight Magnitude` parameter. This parameter defines how diffciult the given cryptographic puzzle should be to be accepted by the network. As of now, you should set at least `min_weight_magnitude=14` in case of mainnet.

Please note, POW is performed per each transaction and so it can take some time. That's why it is not recommended to have more than 30 transactions in a bundle.

Once succesfully performed, the modified bundle (list of all transactions in trytes) is returned in order to be broadcasted.

Specifically:
* Fields `nonce` and `transaction hash` are calculated per each transaction
* `Trunk` and `Branch` tips are correctly mapped among transactions within the bundle. Check that `trunk` of the first TX refers to the second TX, etc.

In [8]:
print("Performing POW... Wait please...\n")
att = api.attach_to_tangle(trunk_transaction=gta['trunkTransaction'], # first tip selected
                           branch_transaction=gta['branchTransaction'], # second tip selected
                           trytes=Trytes, # our finalized bundle in Trytes
                           min_weight_magnitude=14) # MWMN
pprint(att)

Performing POW... Wait please...

{'duration': 12267,
 'fieldName': 'iota-fn01.sairai.de',
 'fieldPublicId': '45f52c5a2e829a00',
 'fieldVersion': '0.1.6',
 'trytes': [TransactionTrytes(b'RBTCFDTCEARCCDADTCGDEAPCEAUCXCFDGDHDEAADTCGDGDPCVCTCSAEAXBCDKDEAXCGDEAWAUAVABBRAUA9BRAWAABEAUA9BDBWAZADBZACBSAXAVAABXAABWA9999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999

You can eventually also preview what specific fields have been modified using transaction objects. The bundle is ready to be broadcasted and so it also show you all fields/values that will be stored in the network.

In [10]:
# show what has been broadcasted - hash transaction + nonce (POW)
print("Final bundle including POW and branch/trunk transactions:\n")
for t in att['trytes']:
    pprint(vars(iota.Transaction.from_tryte_string(t)))
    print("")

Final bundle including POW and branch/trunk transactions:

{'_legacy_tag': Tag(b'PVIBEK999IOTA999TUTORIAL999'),
 'address': Address(b'CXDUYK9XGHC9DTSPDMKGGGXAIARSRVAFGHJOCDDHWADLVBBOEHLICHTMGKVDOGRU9TBESJNHAXYPVJ9R9'),
 'attachment_timestamp': 1530081729891,
 'attachment_timestamp_lower_bound': 0,
 'attachment_timestamp_upper_bound': 12,
 'branch_transaction_hash': TransactionHash(b'RRYDZLPS9IWCSWRROPQQJTKBRONOJNLUHVHFXFEOZAVGFEOUAHXWWHJSHAGWSSNSMWHN9SEYNBOEA9999'),
 'bundle_hash': BundleHash(b'TL9EHBGBIDCTLCARLPKBSUQIG9CKQPDGTLOVNLG9TALAGEWQRBEK9DPLDDBGOAZIREKGYPYFCWJCTOADW'),
 'current_index': 0,
 'hash': TransactionHash(b'SYDECKQB9BHWFUYOZMANREHOCXHCOPY9JCSZSNIZRNDJTZXHHSESCCLQOENGRVFLDMNXYAHHEWRDZ9999'),
 'is_confirmed': None,
 'last_index': 1,
 'nonce': Nonce(b'QRZIERPEJFLKTIVKEBQIHJXTAX9'),
 'signature_message_fragment': Fragment(b'RBTCFDTCEARCCDADTCGDEAPCEAUCXCFDGDHDEAADTCGDGDPCVCTCSAEAXBCDKDEAXCGDEAWAUAVABBRAUA9BRAWAABEAUA9BDBWAZADBZACBSAXAVAABXAABWA9999999999999999999999999999

#### Broadcasting
Now it is time to broadcast the given bundle to the network. List of all transations in trytes returned from the previous step is only thing needed.

Once succesfully broadcasted it returns the same input bundle as a confirmation.

In [11]:
print("Broadcasting transaction...")
res = api.broadcast_and_store(att['trytes'])
pprint(res)

Broadcasting transaction...
{'trytes': [TransactionTrytes(b'RBTCFDTCEARCCDADTCGDEAPCEAUCXCFDGDHDEAADTCGDGDPCVCTCSAEAXBCDKDEAXCGDEAWAUAVABBRAUA9BRAWAABEAUA9BDBWAZADBZACBSAXAVAABXAABWA99999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999

You can check the broadcasted bundle via The Tangle Explorer at any of the receiving addresses:
* Target address 1: https://thetangle.org/address/CXDUYK9XGHC9DTSPDMKGGGXAIARSRVAFGHJOCDDHWADLVBBOEHLICHTMGKVDOGRU9TBESJNHAXYPVJ9R9
* Target address 2: 
https://thetangle.org/address/CYJV9DRIE9NCQJYLOYOJOGKQGOOELTWXVWUYGQSWCNODHJAHACADUAAHQ9ODUICCESOIVZABA9LTMM9RW

### Creating and broadcasting transaction in a single call (you want send a transaction quickly)
As mentioned above, PyOTA library (and almost any other IOTA-facing library) is able to encapsulate all implementation details and so you do not need to care of them anymore. It is usually based on *extended API calls* that were proposed to be included by IOTA Foundation to make developer's life easier (https://github.com/iotaledger/wiki/blob/master/api-proposal.md).

Even if using extended API calls you can still slightly control a level of your involvement in the whole process. From almost no involvement and fully automatic (*send_transfer()*) to semi-manual approach discussed above.

![Different API calls](https://raw.githubusercontent.com/Hribek25/IOTA101/master/Graphics/Different_API_calls.png "By Petr Zizka")

There are some specifics you should consider while designing your app:
* You are loosing a bit of control of individual steps of the process and you have to rely of some predefined parameters (tip selection algo, etc.)
* If there is an exception raised during the process then you need to go back at beginning and restart the whole process again

So it is basically your call to decide what works best for you.

Now back to our exercise. Here comes a code that basically broadcasts transactions in a single call (*send_transfer()*) and you do not have to care of any implemenation details that are hidden under the hood.

In [16]:
import iota
from datetime import datetime
from pprint import pprint

MySeed = b"HGW9HB9LJPYUGVHNGCPLFKKPNZAIIFHZBDHKSGMQKFMANUBASSMSV9TAJSSMPRZZU9SFZULXKJ9YLAIUA"
TargetAddress1 = b"CXDUYK9XGHC9DTSPDMKGGGXAIARSRVAFGHJOCDDHWADLVBBOEHLICHTMGKVDOGRU9TBESJNHAXYPVJ9R9"
TargetAddress2 = b"CYJV9DRIE9NCQJYLOYOJOGKQGOOELTWXVWUYGQSWCNODHJAHACADUAAHQ9ODUICCESOIVZABA9LTMM9RW"

NowIs = datetime.now() # get a actual date & time - just to have some meaningfull info

# preparing transactions
pt = iota.ProposedTransaction(address = iota.Address(TargetAddress1), # 81 trytes long address
                              message = iota.TryteString.from_unicode('Here comes a first message. Now is %s' % (NowIs)),
                              tag     = iota.Tag(b'HRIBEK999IOTA999TUTORIAL'), # Up to 27 trytes
                              value   = 0)

pt2 = iota.ProposedTransaction(address = iota.Address(TargetAddress2), # 81 trytes long address
                               message = iota.TryteString.from_unicode('Here comes a second message. Now is %s' % (NowIs)),
                               tag     = iota.Tag(b'HRIBEK999IOTA999TUTORIAL'), # Up to 27 trytes
                               value   = 0)
# besides the given attributes, library also adds a transaction timestamp


api = iota.Iota("https://field.carriota.com:443")

print("Preparing/Broadcasting... Wait please...")
# the whole process initiated in a single call
FinalBundle = api.send_transfer(depth=3,
                                transfers=[pt,pt2],
                                min_weight_magnitude=14)['bundle'] # it returns a dictionary with a bundle object
    
#bundle is broadcasted, let's print it
print("\nGenerated bundle hash: %s" % (FinalBundle.hash))
print("\nTail Transaction in the Bundle is a transaction #%s." % (FinalBundle.tail_transaction.current_index))
        
print("\nList of all transactions in the bundle:\n")
for txn in FinalBundle:
    pprint(vars(txn))
    print("")

Preparing/Broadcasting... Wait please...

Generated bundle hash: BUNIIHDTRGQYDBWHCELXHFXLUI9NJEGADGXDHHZIKIRULELTKVPTQJH9LJRVLBNVIIZKZJAGKRPHVQJAB

Tail Transaction in the Bundle is a transaction #0.

List of all transactions in the bundle:

{'_legacy_tag': Tag(b'GVIBEK999IOTA999TUTORIAL999'),
 'address': Address(b'CXDUYK9XGHC9DTSPDMKGGGXAIARSRVAFGHJOCDDHWADLVBBOEHLICHTMGKVDOGRU9TBESJNHAXYPVJ9R9'),
 'attachment_timestamp': 1530093517465,
 'attachment_timestamp_lower_bound': 0,
 'attachment_timestamp_upper_bound': 12,
 'branch_transaction_hash': TransactionHash(b'NEVBYHEKVOARHWETDYOBAJGWYQRTPH9Y9MJSRNPPYYSXGLLQECPLTSTFTJNUMTCQUPXLMYQ99FTNZ9999'),
 'bundle_hash': BundleHash(b'BUNIIHDTRGQYDBWHCELXHFXLUI9NJEGADGXDHHZIKIRULELTKVPTQJH9LJRVLBNVIIZKZJAGKRPHVQJAB'),
 'current_index': 0,
 'hash': TransactionHash(b'FPGUNFXJGLLYRIBBCMPKGZSCPCQLB9XRWLUKEL9QLOGDTRPSNLRMDKFTNH9ASOCKTBK9XV9OTLQLZ9999'),
 'is_confirmed': None,
 'last_index': 1,
 'nonce': Nonce(b'GCFAMCTYWHMOBXYOUAKKJL9AXKV'),
 'signatu

You can check the broadcasted bundle via The Tangle Explorer at any of the receiving addresses:
* Target address 1: https://thetangle.org/address/CXDUYK9XGHC9DTSPDMKGGGXAIARSRVAFGHJOCDDHWADLVBBOEHLICHTMGKVDOGRU9TBESJNHAXYPVJ9R9
* Target address 2: 
https://thetangle.org/address/CYJV9DRIE9NCQJYLOYOJOGKQGOOELTWXVWUYGQSWCNODHJAHACADUAAHQ9ODUICCESOIVZABA9LTMM9RW

And to complete the picture here is addtional code that uses prepare_transfer() and send_trytes() combo.

In [1]:
import iota
from datetime import datetime
from pprint import pprint

MySeed = b"HGW9HB9LJPYUGVHNGCPLFKKPNZAIIFHZBDHKSGMQKFMANUBASSMSV9TAJSSMPRZZU9SFZULXKJ9YLAIUA"
TargetAddress1 = b"CXDUYK9XGHC9DTSPDMKGGGXAIARSRVAFGHJOCDDHWADLVBBOEHLICHTMGKVDOGRU9TBESJNHAXYPVJ9R9"
TargetAddress2 = b"CYJV9DRIE9NCQJYLOYOJOGKQGOOELTWXVWUYGQSWCNODHJAHACADUAAHQ9ODUICCESOIVZABA9LTMM9RW"

NowIs = datetime.now() # get a actual date & time - just to have some meaningfull info

# preparing transactions
pt = iota.ProposedTransaction(address = iota.Address(TargetAddress1), # 81 trytes long address
                              message = iota.TryteString.from_unicode('Here comes a first message. Now is %s' % (NowIs)),
                              tag     = iota.Tag(b'HRIBEK999IOTA999TUTORIAL'), # Up to 27 trytes
                              value   = 0)

pt2 = iota.ProposedTransaction(address = iota.Address(TargetAddress2), # 81 trytes long address
                               message = iota.TryteString.from_unicode('Here comes a second message. Now is %s' % (NowIs)),
                               tag     = iota.Tag(b'HRIBEK999IOTA999TUTORIAL'), # Up to 27 trytes
                               value   = 0)
# besides the given attributes, library also adds a transaction timestamp


api = iota.Iota("https://field.carriota.com:443")

# Creating bundle, preparing inputs and finalizing bundle. It returns trytes of prepared TXs
Trytes = api.prepare_transfer(transfers=[pt,pt2]) 
    
print("Almost prepared bundle - tips and POW are still missing")
pprint(Trytes)
    
print("\nSearching for tips and performing POW... Wait please...")
Result = api.send_trytes(trytes=Trytes["trytes"],
                         depth=3) # Searching for tips, performing POW and broadcasting

print("Bundle was broadcasted.")
  
print("\nFinal transactions were returned - including nonce (POW)")
pprint(Result)

Almost prepared bundle - tips and POW are still missing
{'trytes': [TransactionTrytes(b'RBTCFDTCEARCCDADTCGDEAPCEAGDTCRCCDBDSCEAADTCGDGDPCVCTCSAEAXBCDKDEAXCGDEAWAUAVABBRAUA9BRAXAUAEAUAABDBXAXADBYA9BSAXAYAWAWAZAYA99999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999

You can check the broadcasted bundle via The Tangle Explorer at any of the receiving addresses:
* Target address 1: https://thetangle.org/address/CXDUYK9XGHC9DTSPDMKGGGXAIARSRVAFGHJOCDDHWADLVBBOEHLICHTMGKVDOGRU9TBESJNHAXYPVJ9R9
* Target address 2: 
https://thetangle.org/address/CYJV9DRIE9NCQJYLOYOJOGKQGOOELTWXVWUYGQSWCNODHJAHACADUAAHQ9ODUICCESOIVZABA9LTMM9RW

## Value transactions
Meta transactions were discussed in the previous chapter. This chapter describes `value transactions` which are transactions that transfer IOTA tokens between IOTA addresses. Tokens are fully premined and so number of tokens in the network is given. So `transfering tokens` means changing their ownership only.

While preparing a bundle with `value transations` it is important to distinguish three different transaction types within a bundle:
* **Input TXs:** those are transactions that *deducting* (spending) value from address(es). Those transactions have negative `value` field. They usually refer to `senders`
* **Output TXs:** those are transactions that *adding* (receiving) value to address(es). Those transactions have positive `value` field. They usually refer to `recipients`
* **Unspent/change TX:** technically speaking it is also *output* transaction that *adding* value however it usually refers to `senders` and so it is better to consider it separated from other output TXs

All three types have to be part of each bundle otherwise you are under a risk of loosing IOTA tokens. In other words, each bundle should consists of at least one `input` TX, one `output` TX and one `unspent` TX. Total balance of the bundle should be zero since the number of tokens is the same, just token owners have been "changed".

Example:
* Alice owns 100 tokens (*input = 100*)
* Alice is going to send 20 tokens to John from the address that includes 100 tokens of her (*Alice -100*)
* John is going to receive 20 tokens from Alice (*output = 20*; *John +20*)
* Alice still owns 80 tokens after the transaction (*unspent = 80*; *Alice +80*)

Alice's bundle should look like:
\begin{align}
Input - Output & = \ Unspent \\
Input - Output - Unspent & = \ 0 \\
100 - 20 - 80 & = \ 0
\end{align}

Why Alice has to "put" all 100 tokens in the bundle while she is going to actually spend only 20 tokens? The reason is that IOTA protocol uses one-time signature scheme (OTS) for transaction signing while spending tokens. And with every signing process a half of a private key of the given address is revealed. That's why no one should ever spend tokens from the same address again.
![Never reuse address](https://raw.githubusercontent.com/Hribek25/IOTA101/17cde67e39b66dd7c481577497e08399fc888f96/Graphics/never-reuse-address-again.png "By Petr Zizka")

### Spending tokens
*Please note: for a purpose of this chapter we are going to switch to a test network called `devnet`. It is the testing Tangle. Tokens are not real and everything you do have no implication to the mainnet at all. However it is stil powered by standard node software and so technicaly speaking it behaves like the mainnet. If you need more info regarding devnet please refer to [the official documentation](https://docs.iota.org/introduction/testnet/introduction).*

Everything what has been mentioned in the previous chapter focused on `non-value` transations is still valid also in case of value transactions however there are several additional tasks you should take care of. Needless to say, IOTA libraries (PyOTA and others) are ready to help you and are able to hide all implementation details in case you would like to. In this case I would encourage everyone to prefer a functionality that is offered by extended API calls, such as `send_transfer`. Experimenting with value transactions and trying to tweak them does not seem to be a good idea (on the mainnet).

Anyway, even in this chapter I am going to describe main tasks that are performed under the hood to be able to understand the whole concept.

Let's have the following setup:

In [1]:
import iota
from pprint import pprint

# Sender's side ***********
# Sender's seed
SeedSender = b"HGW9HB9LJPYUGVHNGCPLFKKPNZAIIFHZBDHKSGMQKFMANUBASSMSV9TAJSSMPRZZU9SFZULXKJ9YLAIUA"

# Receiver's side ***********
# Recipient's seed - this is actually not needed for the exercise - it is just to have corresponding seed somewhere written
SeedReceiver = b"NVPDKGLESTYNJEH9DTWD9DUBYWHZJDDWTBLKWEYLVZKBKKAZTIZ9CFFJMHPEKIFUWQTXRGAVRXAQZCPHL"

# This is our target address to which we are going to send tokens to (BTW: it is a first generated address based on recipient's seed)
AddressReceiver = b"BMFSMZMNBGKHAWPIZIOMJGRBXZETVSAYDSTDQCHLYTBWZMIXLNXF9XHLTMOCATFVFOMBQF9IOQGPEBPDC"

#DevNet node - Devnet Tangle that is used only for testing/dev purposes - no real tokens here:(
DevnetNode = "https://nodes.devnet.iota.org:443"

You may be wondering why only the seed for sender's side is defined. As mentioned above the reason is that the address with tokens is changing with every spending exercise and so you have to find address with tokens that left in a bucket programatically (if any tokens left:).

Let's find out. For this purpose you can use extended API call `get_account_data` which returns a list of used addresses and total sum of all tokens available accross all of them:

In [2]:
api = iota.Iota(DevnetNode,
                seed=SeedSender # let's use seed of the sender and so library can do all hard work
               )

print("Checking for total balance. This may take some time...")

# now we can find out whether there are any tokens left
SenderBalance = api.get_account_data(start=0,
                                    stop=None) # Get the total available balance for the given seed and all related used addresses

print("\nPlease note, this may take some time if many addresses has been used already...")
pprint(SenderBalance)

if SenderBalance['balance']>0:
    print("\nYes, there are some tokens available! hurray")


Please note, this may take some time if many addresses has been used already...
{'addresses': [Address(b'CXDUYK9XGHC9DTSPDMKGGGXAIARSRVAFGHJOCDDHWADLVBBOEHLICHTMGKVDOGRU9TBESJNHAXYPVJ9R9'),
               Address(b'CYJV9DRIE9NCQJYLOYOJOGKQGOOELTWXVWUYGQSWCNODHJAHACADUAAHQ9ODUICCESOIVZABA9LTMM9RW'),
               Address(b'UJCAVPMFHTAHMTVKGCRBKEKXYRPAFFLGHLAWHISFLWUPEXXQAWRZUSFMYPPTIUANFQMKPLTTCLAWDIUY9'),
               Address(b'WMPTNPCWTVBYSEBJUCRZZIINYEUCDCTWWRVZBJPMYUPRFEEJRRDUJGXE9NYGGZMSXJUJHUISFSCWGPLJZ'),
               Address(b'IFLULNLIIGWGV9OBDBZT9JHPVLITVOMRMAUOCGPGMIIEQABQANJTBQ9PLPSFRR9BTCOJGE9ZWIZAXCRAW'),
               Address(b'LHPLHEZNT99XRSCPMQ9XOCIRUYOKVBLTNUVQPZCTMWOM9WHVYPFKGTFQ9MNEPSYKIV9MNPNEKLMLZONHW')],
 'balance': 950,
 'bundles': [<iota.transaction.base.Bundle object at 0x7f5c6aac84e0>,
             <iota.transaction.base.Bundle object at 0x7f5c6ab7b2b0>]}

Yes, there are some tokens available! hurray


Are there any tokens left (value in `balance` field)? If yes then continue - no action is needed.
If not, then you need to obtain some. It can be done visiting [IOTA faucet](http://faucet.devnet.iota.org) and entering an unused address of the given seed:

In [3]:
# Now let's find a first unused address that can be used as a destination address for unspent/new tokens
FreeAddressSender = api.get_new_addresses(index=0,
                                          count=None, # If None is specified, then it identifies a first non-used address
                                          security_level=2) # Let's generate an address using default security level=2.

print("\nPlease note, this may take some time if many addresses has been used already...")
print("\nThis is the first unused address that can be used for new tokens or unspent tokens:")
pprint(FreeAddressSender)



Please note, this may take some time if many addresses has been used already...

This is the first unused address that can be used for new tokens or unspent tokens:
{'addresses': [Address(b'QDCWBZCONHEXBGNTUBXCUGM9BUEBEZODVNYKZYBJPQXUVHFHMJPMNTYLWDFFMWHH9RIKABAJVRSIE9GWD')]}


What have just happened behind the curtain?
* The PyOTA library started to generating addresses from the `index=0` of the given seed and at the same time it also checked whether there are any transactions registered for each address
* If yes, then in case we are after balances, the balance is saved. In case we are after new unused address then the given address is skipped. First address for which there are not registered transactions is returned
* That's why it may be quite time consuming some time. And now you also understand all issues regarding `zero balances` after the snapshot

You may be also wondering how to get individual balances for each used address? This can be checked using the core API call `get_balances`:

In [4]:
# only if there is any used addresses from the previous steps

# it returns the balances as a list in the same order as the addresses were provided as input.
if len(SenderBalance['addresses'])>0:
    print("\nSome positive balance identified. Individual confirmed balances per used addresses:")
    pprint(api.get_balances(addresses=SenderBalance['addresses'],
                           threshold=100) # Confirmation threshold; official docs recommend to set it to 100
           )
else:
    print("\nNo positive balance identified.")


Some positive balance identified. Individual confirmed balances per used addresses:
{'balances': [0, 0, 0, 0, 0, 950],
 'duration': 57,
 'milestone': None,
 'milestoneIndex': 640346,
 'references': ['FDQMYTZBOHGTMBQUTZEK9ZOOHBBEGAETYUIPDFAAQKSJQDMBKSY9VAGSIXR9NWLEUGIOQWXERFZDJR999']}


It returns `balances`. You can see how the remaining tokens move to new unused address everytime you spend some tokens. So let's spend some tokens:

In [5]:
tx1 = iota.ProposedTransaction( address = iota.Address(AddressReceiver), # 81 trytes long address
                                message = None,
                                tag     = iota.Tag(b'HRIBEK999IOTA999TUTORIAL'),
                                value   = 10) # we are going to spend 10i

# Sending the iotas
print("\nSending iotas... Please wait...")
SentBundle = api.send_transfer(depth=3,
                         transfers=[tx1], #one transaction defined above
                         inputs=None, #input address(es) that are used to finance the given transaction. Tokens will be deducted from them
                         change_address=None, # this is the adddress to which unspent amount is sent
                         min_weight_magnitude=9, # in case of Devnet it is 9
                         security_level=2)

# you may be wondering why there is imputs=None and change_address=None.
# It means it will be taken care of by library and so make sure correct seed was added while api initialization

print("\nIt has been sent. Now let's see transactions that were sent:")
# let's check transactions that were sent in fact

print("Here is the bundle hash: %s" % (SentBundle['bundle'].hash))
for tx in SentBundle['bundle']:
    print("\n")
    pprint(vars(tx))




Sending iotas... Please wait...

It has been sent. Now let's see transactions that were sent:
Here is the bundle hash: <iota.transaction.base.Bundle object at 0x7f5c6e0a9160>


{'_legacy_tag': Tag(b'QCIBEK999IOTA999TUTORIAL999'),
 'address': Address(b'BMFSMZMNBGKHAWPIZIOMJGRBXZETVSAYDSTDQCHLYTBWZMIXLNXF9XHLTMOCATFVFOMBQF9IOQGPEBPDC'),
 'attachment_timestamp': 1530694198428,
 'attachment_timestamp_lower_bound': 0,
 'attachment_timestamp_upper_bound': 3812798742493,
 'branch_transaction_hash': TransactionHash(b'XJCTUNEXSGGGW9ZJUNENOFPIXJOETZLNTCRRPISNFFMGTXYQGNYAHLZDILFPI9ABUTZIFBZUISBAA9999'),
 'bundle_hash': BundleHash(b'PWNCLFFBZNWEOGFCDFL9LIZNBPUPBEFZTGLYSSSJTDSIRHZAYJRCWGCDDSKYXIIRKRFNVUF9LWDVOFGGY'),
 'current_index': 0,
 'hash': TransactionHash(b'MFFVLR9SFFZLGNRJMUHZSNZAGLDELLGOOHQI9AUMNIKAWJPEIAHQMEGMNDPDIZDZJAWVJ9LTQJHVHL999'),
 'is_confirmed': None,
 'last_index': 3,
 'nonce': Nonce(b'LARNNZHRUPXZO9OLZAIKLLBAGCL'),
 'signature_message_fragment': Fragment(b'99999999999999999999



https://devnet.thetangle.org/address/BMFSMZMNBGKHAWPIZIOMJGRBXZETVSAYDSTDQCHLYTBWZMIXLNXF9XHLTMOCATFVFOMBQF9IOQGPEBPDC